# Nova formulação

Uma nova formulação a partir do artigo

*Duxbury, Lavor, Liberti, Salles-Neto, Unassigned distance geometry and molecular conformation problems, Journal of Global Optimization, v.83, pp: 73-82, 2022.*

A ideia é substituir a função objetivo do modelo (3) no artigo acima por:
$$
    \min \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} \left( \sum_{k=1}^{m} a_{ij}^k \big\vert \Vert x_i - x_j \Vert_2 - d_k \big\vert \right),
$$
onde $x_i = (x_{i,1}, x_{i,2}, x_{i,3})^\mathsf{T}$, $i = 1, 2, \ldots, n$.

In [5]:
%pip install gurobipy
%pip install py3Dmol

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
import gurobipy as gp
from gurobipy import GRB

## Preprocessamento

In [7]:
import numpy as np

# Lista de distâncias
n = 4
d = np.sort(
    np.array(
        [
            1.5,
            2.5244,
            3.8724,
            1.5,
            2.5244,
            1.5,
        ]
    )
)
m = len(d)

alfa = 0.2


Sejam
$$
    D = \max_{k=1,\ldots, m} \{ d_k \} \quad\text{e}\quad S = \sum_{k=1}^m d_k.
$$

In [8]:
D = d.max()
S = d.sum()


## Modelo

### Variáveis

In [9]:
model = gp.Model("UnassignedDistance")
model.setParam(GRB.Param.NonConvex, 2)

# Variável com as coordenadas
x = {
    i: model.addMVar(3, name=f"x[{i}]", vtype=GRB.CONTINUOUS)
    for i in range(n)
}

# Vetor de distância entre os átomos i e j
v = {
    (i, j): model.addMVar(3, name=f"v[{i},{j}]", vtype=GRB.CONTINUOUS)
    for i in range(n - 1)
    for j in range(i + 1, n)
}

# Distância entre os átomos i e j
w = {
    (i, j): model.addVar(name=f"w[{i},{j}]", vtype=GRB.CONTINUOUS)
    for i in range(n - 1)
    for j in range(i + 1, n)
}

# Variável de decisão (distância k é referente ao par i, j.)
a = {
    (i, j, k): model.addVar(name=f"a[{i},{j},{k}]", vtype=GRB.BINARY)
    for i in range(n - 1)
    for j in range(i + 1, n)
    for k in range(m)
}

# Variável z (distância entre os átomos i e j se distância k é referente ao par i, j. Zero em caso contrário.)
z = {
    (i, j, k): model.addVar(name=f"z[{i},{j},{k}]", vtype=GRB.CONTINUOUS)
    for i in range(n - 1)
    for j in range(i + 1, n)
    for k in range(m)
}


Restricted license - for non-production use only - expires 2023-10-25
Set parameter NonConvex to value 2


### Função objetivo

O novo modelo:
$$
    \text{(NP):} \quad \min \sum_{i=1}^{n-1} \sum_{j=i+1}^{n} \left( \sum_{k=1}^{m} z_{ijk} \right) - S,
$$

In [10]:
model.setObjective(
    gp.quicksum(
        z[i, j, k] for i in range(n - 1) for j in range(i + 1, n) for k in range(m)
    )
    - S,
    GRB.MINIMIZE,
)


### Restrições

O problema está sujeito a:
$$
\begin{aligned}
    &\text{(C1):} & z_{ijk} &= a_{ij}^k \Vert x_{i} - x_{j} \Vert_2 \\
    &\text{(C2):} & -a_{ij}^k D &\leq z_{ijk} \leq a_{ij}^k D \\
    &\text{(C3):} & d_k - (1-a_{ij}^k) D &\leq z_{ijk} \leq d_k + (1-a_{ij}^k) D 
\end{aligned}
$$
para $i = 1, 2, \ldots, n−1,\; j = i+1, i+2, \ldots, n,\; k = 1, 2, \ldots, m$

In [11]:
for i in range(n - 1):
    for j in range(i + 1, n):

        model.addConstr(v[i, j] == x[i] - x[j], name=f"C1auxA_{i}_{j}")
        model.addConstr(w[i, j] == gp.norm(v[i, j], 2), name=f"C1auxB_{i}_{j}")

        for k in range(m):
            # C1
            model.addConstr(z[i, j, k] == a[i, j, k] * w[i, j], name=f"C1_{i}_{j}_{k}")
            # C2A
            model.addConstr(z[i, j, k] <= a[i, j, k] * D, name=f"C2A_{i}_{j}_{k}")
            # C2B
            model.addConstr(z[i, j, k] >= -a[i, j, k] * D, name=f"C2B_{i}_{j}_{k}")
            # C3A
            model.addConstr(
                z[i, j, k] <= (d[k] + alfa) + (1 - a[i, j, k]) * D,
                name=f"C3A_{i}_{j}_{k}",
            )
            # C3B
            model.addConstr(
                z[i, j, k] >= (d[k] - alfa) - (1 - a[i, j, k]) * D,
                name=f"C3B_{i}_{j}_{k}",
            )


Mantendo as restrições do modelo (3):
$$
\begin{aligned}
    &\text{(C4):} & \sum_{i=1}^{n-1} \sum_{j=1+1}^{n} a_{ij}^k &= 1 && k = 1, 2, \ldots, m, \\
    &\text{(C5):} & \sum_{k=1}^{m} a_{ij}^k &\leq 1 && i = 1, 2, \ldots, n−1,\; j = i+1, i+2, \ldots, n,
\end{aligned}
$$

In [12]:
# C4
c5 = model.addConstrs(
    gp.quicksum(a[i, j, k] for i in range(n - 1) for j in range(i + 1, n)) == 1
    for k in range(m)
)
# C5
c6 = model.addConstrs(
    gp.quicksum(a[i, j, k] for k in range(m)) <= 1
    for i in range(n - 1)
    for j in range(i + 1, n)
)


## Solução

In [13]:
# Salva a formulação do modelo
model.write("unassigned_distance.lp")

# Otimiza o modelo
model.setParam("TimeLimit", 5 * 60)
model.setParam("LogToConsole", 0)

model.optimize()


Set parameter TimeLimit to value 300


In [14]:
# Salva os resultados no arquivo out.sol
model.write("out.sol")


## Pós-processamento

In [19]:
def gurobi_variables_to_xyz_str(x: list):
    xyz_coords = [f"C   {x_i[0].X:.4f}   {x_i[1].X:.4f}   {x_i[2].X:.4f}" for x_i in x.values()]
    return "\n".join([str(n), "OUTPUT", *xyz_coords])


In [20]:
xyz_str = gurobi_variables_to_xyz_str(x)

with open("output.xyz", "w") as f:
    f.write(xyz_str)

print(xyz_str)

4
OUTPUT
C   3.3010   1.0094   1.2533
C   2.0348   0.8879   0.9852
C   1.2660   0.1227   0.2687
C   0.0000   0.0000   0.0000


In [21]:
import py3Dmol
view = py3Dmol.view(data=xyz_str)
view.setBackgroundColor('000')
view.setStyle({'stick':{}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol